In [44]:
import numpy as np
import pandas as pd
import math
import requests
import xlsxwriter
from tqdm import tqdm


In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")

In [3]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [4]:
stocks.iloc[1][0]

'AAL'

In [5]:
from secrets import IEX_CLOUD_API_TOKEN

In [6]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"

data = requests.get(api_url).json()

price = data["latestPrice"]
marketCap = data["marketCap"]


In [7]:
my_colunms = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_colunms)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [8]:
final_dataframe.append(
    pd.Series([symbol, price, marketCap, 'N/A'],index = my_colunms),ignore_index = True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,137.05,2200497528869,N/A


In [9]:
final_dataframe = pd.DataFrame(columns=my_colunms)

In [10]:
error_log = []

In [11]:
for i in tqdm(range(stocks.shape[0])):
    try:
        symbol = stocks.iloc[i][0]
        api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
        data = requests.get(api_url).json()
        price = data['latestPrice']
        marketCap = data['marketCap']

        stock_series = pd.Series([symbol, price, marketCap, 'N/A'], index=my_colunms)
        final_dataframe = final_dataframe.append(stock_series, ignore_index=True)
    except:
        error_log.append(symbol)

100%|████████████████████████████████████████████████████████████████████████████████| 505/505 [06:22<00:00,  1.32it/s]


In [22]:
def assign_stock_groups(stocks, group_size):
    for i in range(0, len(stocks), group_size):
        yield stocks[i:i+group_size]

In [23]:
stock_groups = list(assign_stock_groups(stocks["Ticker"], 100))

In [24]:
stock_string = []
for i in range(0, len(stock_groups)):
    stock_string.append(','.join(stock_groups[i]))
stock_string

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [25]:
final_dataframe = pd.DataFrame(columns=my_colunms)

## Batch

In [28]:
for stock_string in stock_string:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={stock_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for stock in stock_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                stock, 
                data[stock]['quote']['latestPrice'], 
                data[stock]['quote']['marketCap'],
                'N/A'
            ], 
            index = my_colunms),
            ignore_index=True
        )
final_dataframe
    

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,117.80,35284421993,N/A
1,AAL,13.50,8822976909,N/A
2,AAP,182.36,10881262923,N/A
3,AAPL,141.92,2290008456229,N/A
4,ABBV,151.80,270222766472,N/A
...,...,...,...,...
131,DGX,135.80,16505045407,N/A
132,DHI,65.26,23079910103,N/A
133,DHR,246.29,184608798555,N/A
134,DIS,96.10,176617229801,N/A


In [37]:
portfolio_size = input('Enter the value of your porfolio: ')
val = float(portfolio_size)


Enter the value of your porfolio: 1000000


In [40]:
position_size = val/len(final_dataframe.index)
print(position_size)

for i in range(0, len(final_dataframe.index)):
    #Many place does not support fractional trading
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
    
final_dataframe


7352.941176470588


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,117.80,35284421993,62
1,AAL,13.50,8822976909,544
2,AAP,182.36,10881262923,40
3,AAPL,141.92,2290008456229,51
4,ABBV,151.80,270222766472,48
...,...,...,...,...
131,DGX,135.80,16505045407,54
132,DHI,65.26,23079910103,112
133,DHR,246.29,184608798555,29
134,DIS,96.10,176617229801,76


In [45]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [47]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [48]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [49]:
writer.save()